In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # 시각화 라이브러리
import warnings # 경고창 없애기
import seaborn as sns
warnings.filterwarnings('ignore')

# 

# 데이터 수집

In [6]:
# 데이터 전처리
X = pd.read_csv('./maemaefilter.csv')

# 데이터 전처리

In [7]:
X.head()
# 매매번호, 법정동, 아파트, 년, 월, 일 drop

,매매번호,법정동,거래금액,건축년도,아파트,년,월,일,평수,층,범죄자수,충전소,지하철,스타벅스,영화관
0,0,중흥동,"15,000",2015,중흥동도나우타운,2020,10,8,18,5,8.0,0,2호선,1.0,메가박스
1,1,중흥동,"34,000",2015,중흥동 영무예다음 아파트,2020,10,10,25,6,8.0,0,2호선,1.0,메가박스
2,2,중흥동,"10,750",1981,평화맨션,2020,10,14,22,4,8.0,0,2호선,1.0,메가박스
3,3,중흥동,"21,800",1991,모아1,2020,10,15,25,4,8.0,0,2호선,1.0,메가박스
4,4,중흥동,"20,000",2017,프라임 아너팰리스,2020,10,16,19,15,8.0,0,2호선,1.0,메가박스


## 결측치 채우기

In [8]:
X['범죄자수'] = X['범죄자수'].fillna(0)
X['스타벅스'] = X['스타벅스'].fillna(0)

## 속성정리

In [9]:
# 매매번호, 법정동, 아파트, 년, 월, 일 drop
X = X.drop(['매매번호', '법정동', '아파트', '년', '월', '일'], axis=1)

## 거래금액 
### 거래금액 쉼표제거, 자료형 Object -> int

In [10]:
X['거래금액'] = X['거래금액'].str.replace(',', '')
X['거래금액'] = X['거래금액'].astype(int)

### y정해주기 y = X['거래금액']

In [11]:
y = pd.DataFrame(X['거래금액'])

###  거래금액 -> price_group

In [12]:
X['거래금액'].value_counts()

15000     306
12000     284
14000     266
16000     262
13000     262
         ... 
153500      1
204000      1
31980       1
54100       1
7050        1
Name: 거래금액, Length: 1545, dtype: int64

In [13]:
X['거래금액'].describe()

count     30826.000000
mean      26377.957666
std       17027.640549
min        3000.000000
25%       14000.000000
50%       22200.000000
75%       34100.000000
max      204000.000000
Name: 거래금액, dtype: float64

In [14]:
# bins : 구간
# 0초과 1이하, 1초과 4이하, 4초과 20이하
bins = [2900, 13000, 15000, 18000, 
        22000, 27000, 35000, 
        40000, 45000, 50000, 55000, 
        60000, 70000, 80000, 90000, 100000, 
        110000, 120000, 130000, 140000, 
        150000, 160000, 180000, 205000]
# labels 구간별 명칭
labels = [10000, 14000, 16500, 20000, 
          24000, 31000, 37500, 
          42500, 47500, 52500, 57500, 
          65000, 75000, 85000, 95000, 105000, 
          115000, 125000, 135000, 145000, 
          160000, 170000, 200000]
# cut 함수 : 카테고리(범주형) 데이터를 만듦 bins => 속성 값 / labels => 범주 이름
X_cut = pd.cut(X['거래금액'], bins = bins, labels = labels)
X['price_group'] = X_cut

## 건축년도

In [15]:
# 1995년 전에는 낡아서 가격이 별 차이 없을거라고 생각함
bins = [1975, 1985, 1995, 2000, 2005, 2010, 2015, 2020, 2025]
labels = [1, 500, 1000, 1500, 2000, 2500, 3000, 3500]
X_cut = pd.cut(X['건축년도'], bins = bins, labels = labels)
X['arch_group'] = X_cut

## 평수

In [16]:
# bins :구간
# 0 초과 1이하, 1초과 4이하 ~
bins = [0, 6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 85]
labels = [1, 500, 1000, 1500, 2500, 3000, 3500, 4500, 5000, 5500, 6000, 6500, 7000, 7500]
X_cut = pd.cut(X['평수'], bins = bins, labels = labels)
X['area_group'] = X_cut

## 층

In [17]:
# bins : 구간
# 0초과 1이하, 1초과 4이하, 4초과 20이하
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
# labels 구간별 명칭
labels = [1, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500]
# cut 함수 : 카테고리(범주형) 데이터를 만듦 bins => 속성 값 / labels => 범주 이름
X_cut = pd.cut(X['층'], bins = bins, labels = labels)
X['layer_group'] = X_cut

## 그룹화한 컬럼들 지우기, 그룹화자료형 : 카테고리형 > 정수형

In [18]:
# 추가한 컬럼 남기고 기존 컬럼 지우기
X = X.drop(['거래금액', '건축년도', '평수', '층'], axis = 1)

In [19]:
X['price_group'] = X['price_group'].astype(int)
X['arch_group'] = X['arch_group'].astype(int)
X['area_group'] = X['area_group'].astype(int)
X['layer_group'] = X['layer_group'].astype(int)

## 범죄자수 자료형 float-> int

In [20]:
X['범죄자수'] = X['범죄자수'].astype(int)

In [21]:
X['범죄자수'].value_counts()

0    12766
1     8066
2     3923
4     3154
3     1304
5     1235
8      378
Name: 범죄자수, dtype: int64

In [22]:
# bins : 구간
# 0초과 1이하, 1초과 4이하, 4초과 20이하
bins = [-1, 0, 1, 2, 3, 4, 5, 8]
# labels 구간별 명칭
labels = [0, -500, -1000, -1500, -2000, -2500, -3000]
# cut 함수 : 카테고리(범주형) 데이터를 만듦 bins => 속성 값 / labels => 범주 이름
X_cut = pd.cut(X['범죄자수'], bins = bins, labels = labels)
X['criminal_group'] = X_cut
X['criminal_group'] = X['criminal_group'].astype(int)

In [23]:
X['criminal_group'].value_counts()

 0       12766
-500      8066
-1000     3923
-2000     3154
-1500     1304
-2500     1235
-3000      378
Name: criminal_group, dtype: int64

In [24]:
# crime_map = {
#     0:0,
#     1:-1,
#     2:-2,
#     3:-3,
#     4:-4,
#     5:-5,
#     8:-8
# }

In [25]:
# X['범죄자수'] = X['범죄자수'].map(crime_map, X['범죄자수'])

## 스타벅스 자료형 float-> int

In [26]:
# bins : 구간
# 0초과 1이하, 1초과 4이하, 4초과 20이하
bins = [-1, 0, 1]
# labels 구간별 명칭
labels = [0, 4000]
# cut 함수 : 카테고리(범주형) 데이터를 만듦 bins => 속성 값 / labels => 범주 이름
X_cut = pd.cut(X['스타벅스'], bins = bins, labels = labels)
X['star_group'] = X_cut
X['star_group'] = X['star_group'].astype(int)

In [27]:
X['star_group'].value_counts()

4000    16839
0       13987
Name: star_group, dtype: int64

## 텍스트 데이터 지하철, 영화관
### 지하철 매핑으로 숫자로 바꿔주기

In [28]:
sub_map = {
    '2호선':2,
    '0':0,
    '1호선':1
}

In [29]:
X['지하철'] = X['지하철'].map(sub_map, X['지하철'])

In [30]:
X['지하철'].value_counts()

2    15115
0    13178
1     2533
Name: 지하철, dtype: int64

In [31]:
# bins : 구간
# 0초과 1이하, 1초과 4이하, 4초과 20이하
bins = [-1, 0, 1, 2]
# labels 구간별 명칭
labels = [-2000, 1000, 4000]
# cut 함수 : 카테고리(범주형) 데이터를 만듦 bins => 속성 값 / labels => 범주 이름
X_cut = pd.cut(X['지하철'], bins = bins, labels = labels)
X['subway_group'] = X_cut
X['subway_group'] = X['subway_group'].astype(int)

In [32]:
X['subway_group'].value_counts()

 4000    15115
-2000    13178
 1000     2533
Name: subway_group, dtype: int64

### 영화관 매핑으로 0~ 2점 주기

In [33]:
theater_map = {
    '0':0,
    'CGV':1,
    '메가박스':1,
    '롯데시네마':1,
    '메가박스, 롯데시네마':2,
    '메가박스, CGV':2
}

In [34]:
X['영화관'] = X['영화관'].map(theater_map, X['영화관'])

In [35]:
X['영화관'].value_counts()

0    27233
1     1996
2     1597
Name: 영화관, dtype: int64

In [36]:
# bins : 구간
# 0초과 1이하, 1초과 4이하, 4초과 20이하
bins = [-1, 0, 1, 2]
# labels 구간별 명칭
labels = [0, 500, 1000]
# cut 함수 : 카테고리(범주형) 데이터를 만듦 bins => 속성 값 / labels => 범주 이름
X_cut = pd.cut(X["영화관"], bins = bins, labels = labels)
X['movie_group'] = X_cut
X['movie_group'] = X['movie_group'].astype(int)

In [37]:
X['movie_group'].value_counts()

0       27233
500      1996
1000     1597
Name: movie_group, dtype: int64

### 전기차 충전소 값 주기

In [38]:
X['충전소'].value_counts()

0    26422
1     4404
Name: 충전소, dtype: int64

In [39]:
# bins : 구간
# 0초과 1이하, 1초과 4이하, 4초과 20이하
bins = [-1, 0, 1]
# labels 구간별 명칭
labels = [0, 4000]
# cut 함수 : 카테고리(범주형) 데이터를 만듦 bins => 속성 값 / labels => 범주 이름
X_cut = pd.cut(X['충전소'], bins = bins, labels = labels)
X['charge_group'] = X_cut
X['charge_group'] = X['charge_group'].astype(int)

In [40]:
X['charge_group'].value_counts()

0       26422
4000     4404
Name: charge_group, dtype: int64

In [41]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30826 entries, 0 to 30825
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   범죄자수            30826 non-null  int32  
 1   충전소             30826 non-null  int64  
 2   지하철             30826 non-null  int64  
 3   스타벅스            30826 non-null  float64
 4   영화관             30826 non-null  int64  
 5   price_group     30826 non-null  int32  
 6   arch_group      30826 non-null  int32  
 7   area_group      30826 non-null  int32  
 8   layer_group     30826 non-null  int32  
 9   criminal_group  30826 non-null  int32  
 10  star_group      30826 non-null  int32  
 11  subway_group    30826 non-null  int32  
 12  movie_group     30826 non-null  int32  
 13  charge_group    30826 non-null  int32  
dtypes: float64(1), int32(10), int64(3)
memory usage: 2.1 MB


## 맞춤필터도 범위형으로 값 주기

In [42]:
X = X.drop(['범죄자수', '충전소', '지하철', '스타벅스', '영화관'], axis = 1)

# EDA 탐색적데이터분석

# 모델선택 및 하이퍼파라미터 조정

In [43]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ridge_regression

## 특성확장 X

## train, test 분리

In [44]:
# X_train = X[:23000]
# X_test = X[23000:]
# y_train = y[:23000]
# y_test = y[23000:]

In [45]:
X.shape, y.shape

((30826, 9), (30826, 1))

In [46]:
X

,price_group,arch_group,area_group,layer_group,criminal_group,star_group,subway_group,movie_group,charge_group
0,14000,2500,1000,1,-3000,4000,4000,500,0
1,31000,2500,2500,500,-3000,4000,4000,500,0
2,10000,1,1500,1,-3000,4000,4000,500,0
3,20000,500,2500,1,-3000,4000,4000,500,0
4,20000,3000,1500,1000,-3000,4000,4000,500,0
...,...,...,...,...,...,...,...,...,...
30821,47500,3000,1500,1500,0,4000,-2000,0,0
30822,65000,3000,2500,2500,0,4000,-2000,0,0
30823,65000,3000,2500,1500,0,4000,-2000,0,0
30824,75000,3000,3000,500,0,4000,-2000,0,0


In [47]:
X.keys()

Index(['price_group', 'arch_group', 'area_group', 'layer_group',
       'criminal_group', 'star_group', 'subway_group', 'movie_group',
       'charge_group'],
      dtype='object')

### LinearRegression

linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

print('train score : ', linear_model.score(X_train, y_train))
print('test score : ', linear_model.score(X_test, y_test))

y_test = linear_model.predict(X_test).astype(int)
y_test = pd.DataFrame(y_test)
y_test.to_csv('LinearPrice.csv', index = False)

### Lasso

In [48]:
# lasso_model = Lasso(alpha=2)
# lasso_model.fit(X_train, y_train)

# print('train score : ', lasso_model.score(X_train, y_train))
# print('test score : ', lasso_model.score(X_test, y_test))

# y_test = lasso_model.predict(X_test).astype(int)
# y_test = pd.DataFrame({'거래금액': y_test})
# y_test.to_csv('LassoPrice2.csv', index = True)

## 특성확장

In [49]:
# 특성 확장
extended_X = X.copy()
for col1 in X.columns: 
    for col2 in X.columns: 
        extended_X[col1+'x'+col2] = X[col1] * X[col2]

## train, test 분리

In [50]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(extended_X, y, test_size = 0.3, random_state=25)

In [51]:
X_train = extended_X[:23000]
X_test = extended_X[23000:]
y_train = y[:23000] 
y_test = y[23000:]

In [52]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23000, 90), (7826, 90), (23000, 1), (7826, 1))

## Lasso - extend

In [53]:
lasso_model = Lasso(alpha=2)
lasso_model.fit(X_train, y_train)

print('train score : ', lasso_model.score(X_train, y_train))
print('test score : ', lasso_model.score(X_test, y_test))

y_predict = lasso_model.predict(X_test).astype(int)
y_test = pd.DataFrame({'거래예상금액': y_predict})
y_test.to_csv('ex_LassoPrice.csv', index = True)

train score :  0.9896853913993989
test score :  0.991399397417974


In [54]:
X.info()
# 범죄자 수 1764 개 결측
# 스타벅스 485 개 결측

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30826 entries, 0 to 30825
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   price_group     30826 non-null  int32
 1   arch_group      30826 non-null  int32
 2   area_group      30826 non-null  int32
 3   layer_group     30826 non-null  int32
 4   criminal_group  30826 non-null  int32
 5   star_group      30826 non-null  int32
 6   subway_group    30826 non-null  int32
 7   movie_group     30826 non-null  int32
 8   charge_group    30826 non-null  int32
dtypes: int32(9)
memory usage: 1.1 MB


 -----------------------------------------------------------

# 컬럼별로 데이터프레임 쪼개기
# 컬럼 선택해서 학습, 예측하기

In [55]:
# 1
price = X.loc[:,'price_group':'price_group']
# 2
build_year = X.loc[:,'arch_group':'arch_group']
# 3
apt_size = X.loc[:,'area_group':'area_group']
# 4
floor = X.loc[:,'layer_group':'layer_group']
# 5
crime = X.loc[:,'criminal_group':'criminal_group']
# 6
star = X.loc[:,'star_group':'star_group']
# 7
subway = X.loc[:,'subway_group':'subway_group']
# 8
movie = X.loc[:,'movie_group':'movie_group']
# 9
charge = X.loc[:,'charge_group':'charge_group']

In [56]:
# 2 3 4 5 6 7 8 9
X = pd.concat([build_year, apt_size, floor, crime, star, subway, movie, charge], axis = 1)

## 특성확장

In [57]:
# 특성 확장
extended_X = X.copy()
for col1 in X.columns: 
    for col2 in X.columns: 
        extended_X[col1+'x'+col2] = X[col1] * X[col2]

## train, test 분리

In [58]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(extended_X, y, test_size = 0.3, random_state=25)

In [59]:
X_train = extended_X[:23000]
X_test = extended_X[23000:]
y_train = y[:23000] 
y_test = y[23000:]

In [60]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23000, 72), (7826, 72), (23000, 1), (7826, 1))

In [61]:
X

,arch_group,area_group,layer_group,criminal_group,star_group,subway_group,movie_group,charge_group
0,2500,1000,1,-3000,4000,4000,500,0
1,2500,2500,500,-3000,4000,4000,500,0
2,1,1500,1,-3000,4000,4000,500,0
3,500,2500,1,-3000,4000,4000,500,0
4,3000,1500,1000,-3000,4000,4000,500,0
...,...,...,...,...,...,...,...,...
30821,3000,1500,1500,0,4000,-2000,0,0
30822,3000,2500,2500,0,4000,-2000,0,0
30823,3000,2500,1500,0,4000,-2000,0,0
30824,3000,3000,500,0,4000,-2000,0,0


In [62]:
lasso_model = Lasso(alpha=2)
lasso_model.fit(X_train, y_train)

print('train score : ', lasso_model.score(X_train, y_train))
print('test score : ', lasso_model.score(X_test, y_test))

y_predict = lasso_model.predict(X_test).astype(int)
y_test = pd.DataFrame({'거래예상금액': y_predict})
y_test.to_csv('23456789Price.csv', index = True)

train score :  0.7631660388950987
test score :  0.7236449350802677
